In [127]:
import pickle
import pandas as pd

with open("../data/all_reviews.pkl","rb") as f:
    reviews = pickle.load(f)
    
job_filter = pd.read_csv("../data/filter_job_titles.csv")

job_filters = job_filter.clean_job_title.tolist()


In [128]:
idx = (reviews.language == "en")
reviews = reviews.loc[idx,:]

idx = (reviews.clean_job_title.isin(job_filters))
reviews = reviews.loc[idx,:].reset_index()

print(reviews.shape)
display(reviews.head())

(42785, 16)


,index,company_name,review_title,job_title,employee_status,location,date,review,pros,cons,rating,yes_helpful,no_helpful,industry,clean_job_title,language
0,92,EY,I had a great time working at EY,Software Developer,Former Employee,"Bengaluru, Karnataka",2020-03-02,Pros:\rGreat Learning Curve.\rPeer Culture is ...,NaN,NaN,4.0,0,0,accounting,software developer,en
1,115,EY,Productive and FUN Place,System Engineer,Current Employee,"Hyderabad, Telangana",2020-02-25,It's feel like workplace where can utilize kno...,NaN,NaN,5.0,0,0,accounting,system engineer,en
2,427,EY,Really Productive Gain a lot of knowledge and ...,Network Engineer,Former Employee,Capital Territory,2019-11-13,Enjoy Working There it really help.They Provid...,NaN,NaN,5.0,0,0,accounting,network engineer,en
3,438,EY,A good place to work as fresher,SOFTWARE ENGINEER,Current Employee,"Bengaluru, Karnataka",2019-11-11,"helping team, good work culture, we get to lea...",NaN,NaN,4.0,5,0,accounting,software engineer,en
4,507,EY,Good place to work for Seniors and above,Senior Software Engineer,Former Employee,"Bengaluru, Karnataka",2019-10-21,Good place to work for Seniors and above by co...,NaN,NaN,4.0,0,1,accounting,software engineer,en


In [131]:
reviews[reviews["company_name"]=='Fluor Corp.']

,index,company_name,review_title,job_title,employee_status,location,date,review,pros,cons,rating,yes_helpful,no_helpful,industry,clean_job_title,language
3694,287112,Fluor Corp.,Awesome company,Quality Manager,Former Employee,Afghanistan,2019-10-17,Fluor is a great company with people that care...,NaN,NaN,4.0,0,0,construction,quality manager,en
3695,287958,Fluor Corp.,Was a much better company in the 90's,Control Systems Designer,Former Employee,"Sugar Land, TX",2018-06-06,Fluor was the first powerhouse EPC that went h...,Nice Location,They have no work,3.0,2,0,construction,control systems designer,en
3696,288188,Fluor Corp.,Enjoyed the Work Environment,Senior Software Developer,Current Employee,"Aliso Viejo, CA",2018-03-08,There is not much that I could add here. I en...,Relaxed Environment,None really,5.0,0,1,construction,software developer,en
3697,288687,Fluor Corp.,Good place,System Engineer I,Current Employee,"Piketon, OH",2017-09-15,The place can be pretty boring most of the tim...,NaN,NaN,4.0,0,0,construction,system engineer,en
3698,288949,Fluor Corp.,Hurry up and wait,Software Developer I,Current Employee,"Piketon, OH",2017-06-26,"Everyone is in a hurry to get things done, but...",Great healthcare,Located in the middle of no-where,3.0,1,0,construction,software developer,en
3699,289022,Fluor Corp.,Not what it used to be.,Senior Technical Specialist,Former Employee,"Sugar Land, TX",2017-06-01,Was a re-hire. \rFirst term would rate them a...,Ample parking.,No objective ranking for employees.,3.0,3,0,construction,technical specialist,en
3700,289498,Fluor Corp.,Fluor,Systems Administrator,Former Employee,"Greenville, SC",2016-11-21,I hated working for this company. The pay was ...,NaN,NaN,1.0,2,0,construction,systems administrator,en
3701,289867,Fluor Corp.,Enjoyable work place with social individuals.,it support,Former Employee,"Houston, TX",2015-11-19,Interesting days at work in a enjoyable enviro...,NaN,NaN,5.0,1,0,construction,technology support,en
3702,289895,Fluor Corp.,Good place to work with good benefits,IT Manager,Current Employee,"Denver, CO",2015-10-24,Challenging place to work as IT Manager with f...,NaN,NaN,4.0,3,0,construction,technology manager,en
3703,289947,Fluor Corp.,Happy to work in more relax settings where I c...,Senior Technical Specialist,Former Employee,"Muntinlupa, P00",2015-09-22,Total of thirty-seven (37) years of successful...,Overall management liberal policies,Office politics,4.0,1,0,construction,technical specialist,en


In [2]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [4]:
from nltk import word_tokenize
from nltk.corpus import stopwords

company_name = reviews.company_name
company_name = list(set(company_name))

tokens = map(str.lower, company_name)
tokens = map(str.split, tokens)

stop_words = stopwords.words('english')

print("Stop word count", len(stop_words))

for token in tokens:
    stop_words.extend(token)
    
stop_words = list( set(stop_words) )

print("Stop word count after adding company names", len(stop_words))

Stop word count 179
Stop word count after adding company names 1137


In [118]:
%%time
from multiprocessing import Pool
# from nltk import sent_tokenize
# from nltk.corpus import stopwords
# import spacy
# import re
from nltk.stem.wordnet import WordNetLemmatizer
co_reviews = reviews.review.tolist()
def parseSentence(line):
    result=[]
    lmtzr = WordNetLemmatizer()
    text_token = split_into_sentences(line.lower())
    for i in range(len(text_token)):
        text_rmstop = [lmtzr.lemmatize(word) for word in re.split("\W+",text_token[i]) if word.lower() not in stop_words]
        result.append(' '.join(text_rmstop).strip())
    return result



Wall time: 3.99 ms


In [121]:
tech_review_corpus = list(map(parseSentence, co_reviews))

In [126]:
tech_review_corpus[0]

['pro great learning curve',
 'peer culture great',
 'good work balance',
 'great enthusiasm',
 'view idea considered',
 'con little biased']

In [51]:
from sklearn.feature_extraction.text import CountVectorizer
tech_review_corpus = list(map(parseSentence, co_reviews[1]))

tech_review_corpus[0]

''

In [69]:
text_token = split_into_sentences(co_reviews[1].lower())

lmtzr = WordNetLemmatizer()
#text_token = CountVectorizer().build_tokenizer()(co_reviews[1].lower())
#text_rmstop = [i for i in text_token if i not in stop_words]
#text_stem = [lmtzr.lemmatize(w) for w in text_rmstop]

In [110]:
result=[]
text_rmstop = [word for word in re.split("\W+",text_token[i in range(len(text_token))]) if word.lower() not in stop_words]
result.append(' '.join(text_rmstop).strip())

In [111]:
result

['feel like workplace utilize knowledge skills good supportive teams']

In [107]:
for i in text_token:
    for stword in stop_words:
        text_rmstop = i.replace(stword, "")

In [49]:
lmtzr = WordNetLemmatizer()
text_stem = [lmtzr.lemmatize(w) for w in text_rmstop]

In [68]:

for stword in stop_words:
    print(text_token[0].replace(stword, ""))

it's feel like workplace where can utilize knowledge and skills, very good management and supportive teams.
it's feel like workplace where can utilize knowledge and skills, very good management and supportive teams.
it's feel like workplace where can utilize knowledge and skills, very good management and supportive teams.
it's feel like workplace where can utilize knowledge and skills, very good management and supportive teams.
it's feel like workplace where can utilize knowledge and skills, very good management and supportive teams.
it's feel like workplace where can utilize knowledge and skills, very good management and supportive teams.
it's feel like workplace where can utilize knowledge and skills, very good management and supportive teams.
it's feel like workplace where can utilize knowledge and skills, very good management and supportive teams.
it's feel like workplace where can utilize knowledge and skills, very good management and supportive teams.
it's feel like workpl where 

In [50]:
text_rmstop

["it's  like workplace where can utilize knowledge and skills, very good management and supportive teams.",
 "here have lots to learn and growth have as it's big four company in the word."]